### Deploying models as SQL functions
##### Check that we have correct libraries installed

In [2]:
import tensorflow
import keras
import h5py
assert(tensorflow.__version__ == '1.12.0')
assert(keras.__version__ == '2.2.4')
assert(h5py.__version__ == '2.7.0')

##### Download the flower dataset

In [4]:
%sh
curl -O http://download.tensorflow.org/example_images/flower_photos.tgz
tar xzf flower_photos.tgz &>/dev/null
cd flower_photos
ls
pwd

In [5]:
%sh
rm -rf /databricks/driver/flower_photos/sample_images
cd flower_photos
mkdir sample_images
cp /databricks/driver/flower_photos/daisy/100080576_f52e8ee070_n.jpg sample_images/
cp /databricks/driver/flower_photos/daisy/10140303196_b88d3d6cec.jpg sample_images/
cp /databricks/driver/flower_photos/daisy/10172379554_b296050f82_n.jpg sample_images/

In [6]:
%sh
ls -l /databricks/driver/flower_photos/sample_images

##### Load images into Dataframe

##### Using Spark's ImageSchema

In [9]:
sample_dir = 'file:/databricks/driver/flower_photos/sample_images'

In [10]:
from pyspark.ml.image import ImageSchema
df = ImageSchema.readImages(sample_dir)
df.show()
df.printSchema()

##### Registering SQL User Defined Function

In [12]:
from keras.applications import InceptionV3
from sparkdl.udf.keras_image_model import registerKerasImageUDF

def keras_load_img(fpath):
    from keras.preprocessing.image import load_img, img_to_array
    import numpy as np
    img = load_img(fpath, target_size=(299, 299))
    return img_to_array(img).astype(np.uint8)

registerKerasImageUDF("inceptionV3_udf_with_preprocessing", InceptionV3(weights="imagenet"), keras_load_img)

In [13]:
from pyspark.ml.image import ImageSchema

image_df = ImageSchema.readImages(sample_dir)
image_df.registerTempTable("sample_images")
image_df.show()

In [14]:
%sql 
SELECT inceptionV3_udf_with_preprocessing(image) as predictions from sample_images

##### Decoding the output

In [16]:
len()

In [17]:
import numpy as np
from keras.applications.inception_v3 import preprocess_input, decode_predictions

In [18]:
out = 

In [19]:
out  = np.asarray(out)
print('Predicted:', decode_predictions(out, top=3)[0])

In [20]:
out = out.reshape(-1,1000)
out.shape

In [21]:
print('Predicted: ', decode_predictions(out, top=3)[0])